In [1]:
# dev
profilePath = "s3a://cebu-cdp-data-dev/profiles"
dedupePath = "s3a://cebu-cdp-data-dev/dedupe"
ucpProfilePath = "s3a://cebu-cdp-data-dev/ucp" 
uiPath = "s3a://cebu-cdp-data-dev/ui"
profileMetaPath = "s3a://cebu-cdp-data-dev/last_modified_records"
pathUrlDefault="s3a://edw-dl-process-data-dev/data/odstest/hdata/odstest_all_diffen_open/odstest_"
ciamPath = "s3a://edw-dl-process-data-dev/data/ciam/hdata/ciam_all_diffen_open/ciam_all_edwcustomer"
# qa
# profilePath = "s3a://cebu-cdp-data-qa/profiles"
# dedupePath = "s3a://cebu-cdp-data-qa/dedupe"
# ucpProfilePath = "s3a://cebu-cdp-data-qa/ucp" 
# uiPath = "s3a://cebu-cdp-data-qa/ui"
# profileMetaPath = "s3a://cebu-cdp-data-qa/last_modified_records"
# pathUrlDefault="s3a://edw-dl-process-data-qa/data/odstest/hdata/odstest_all_diffen_open/odstest_"
# ciamPath = "s3://edw-dl-process-data-qa/data/ciam/hdata/ciam_all_diffen_open/ciam_all_edwcustomer"

In [2]:
def entityDetailsFill(entities=[{"entityName": "all_bookingpassenger", "isFact": "true", "isMandatory": "false", "loadLatest": "true", "joinColumn": "BookingID"}], pathUrlDefault=""):
    # build the entity details
    entityDetails = []
    for eachEntity in entities:
        entityDetail = {}
        entityDetail["entityName"] = eachEntity["entityName"]
        entityDetail["entityType"] = "snapshot"  # default
        entityDetail["sourceType"] = "filesystem"  # default
        entityDetail["dataFormat"] = "orc"  # default for diffen
        entityDetail["fileSystem"] = "s3"  # default for aws datalake
        # all fact entites are unioned, preferable to just have one fact entity
        if "isFact" in eachEntity:
            entityDetail["isFact"] = eachEntity["isFact"]
        else:
            entityDetail["isFact"] = "false"
        # profile transformation will fail if mandatory entity is missing
        if "isMandatory" in eachEntity:
            entityDetail["isMandatory"] = eachEntity["isMandatory"]
        else:
            entityDetail["isMandatory"] = "false"
        # load latest version of entity if current date's data snapshot is not available
        if "loadLatest" in eachEntity:
            entityDetail["loadLatest"] = eachEntity["loadLatest"]
        else:
            entityDetail["loadLatest"] = "true"
        # default from entitystore(if entity transformations is executed) or from diffen open folder
        if "pathUrl" in eachEntity:
            entityDetail["pathUrl"] = eachEntity["pathUrl"]
            partitionColumn = "p_date"
        else:
            entityDetail["pathUrl"] = pathUrlDefault + eachEntity["entityName"]
            partitionColumn = "ods"
        entityDetail['partitionDetails'] = {
            "partitionColumn": [
                partitionColumn  # p_date if from entitystore else ods from diffen open folder
            ],
            "partitionType": "daily",
            "partitionFormat": [
                "yyyy-MM-dd"
            ],
            "numberOfPartitions": 1
        }
        entityDetail["joinColumn"] = eachEntity["joinColumn"]
        entityDetails.append(entityDetail)
    return entityDetails

# if your entity data resides in a different bucket other than pathUrlDefault, you can specify the bucket name with the pathUrl for that entity

# isFact default false
# isMandatory default false
# loadLatest default true
entities = [
    {
        "entityName": "all_bookingpassenger",
        "isFact": "true",
        "isMandatory": "true",
        "joinColumn": "BookingID"
    },
    {
        "entityName": "all_booking",
        "isMandatory": "true",
        "joinColumn": "BookingID"
    },
    {
        "entityName": "all_bookingcontact",
        "joinColumn": "BookingID"
    },
    {
        "entityName": "all_passengeraddress",
        "joinColumn": "PassengerID"
    },
    {
        "entityName": "all_passengertraveldoc",
        "joinColumn": "PassengerID"
    },
    {
        "entityName": "all_passengerjourneysegment",
        "joinColumn": "PassengerID"
    },
    {
        "entityName": "all_passengerjourneyleg",
        "joinColumn": "PassengerID"
    },
    {
        "entityName": "all_passengerfee",
        "joinColumn": "PassengerID"
    },
    {
        "entityName": "all_agent",
        "isMandatory": "true",
        "joinColumn": "AgentID"
    },
    {
        "entityName": "all_person",
        "isMandatory": "true",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_personname",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_personemail",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_personphone",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_personaddress",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_traveldoc",
        "joinColumn": "PersonID"
    },
    {
        "entityName": "all_fee",
        "joinColumn": "FeeCode"
    },
    {
        "entityName": "all_payment",
        "joinColumn": "BookingID"
    }

]
print("Number of entities: ", len(entities))
profile_config = {}
profile_config['entityDetails'] = entityDetailsFill(entities=entities, pathUrlDefault = pathUrlDefault)


Number of entities:  17


In [3]:
def columnCompute(columnDetails, primaryKey="ProvisionalPrimaryKey", primaryKeyType="string", defaultValue = "0_0"):
    profileSchema = []
    columnMapping = []
    attributeMapping = []
    profileSchema.append(
        {"columnName": primaryKey, "dataType": primaryKeyType, "defaultValue": defaultValue})

    for eachColumn in columnDetails:
        if eachColumn["name"] != primaryKey:
            profileSchema.append(
                {"columnName": eachColumn["name"], "dataType": eachColumn["type"], "defaultValue": eachColumn["defaultValue"]})
        if eachColumn["skip"] == False:
            if eachColumn['details']['directPull']:
                columnMapping.append({"sourceEntity": eachColumn["details"]['sourceEntity'],
                                      "sourceColumn": eachColumn["details"]['sourceColumn'], "targetColumn": eachColumn["name"]})
            else:
                attributeMapping.append(
                    {"attributeName": eachColumn['name'],
                     "sourceColumns": eachColumn['details']['sourceEntityDetails'],
                     # "joinColumns": [{"entityJoinColumn": eachColumn['details']['eJoin'], "profileJoinColumn": eachColumn['details']['pJoin']}],
                     "transformationFunction": eachColumn['details']['function']
                     })
    return profileSchema, columnMapping, attributeMapping


# bigint - long
# money - double
columnDetails = [
    {
        "name": "PassengerID",
        "type": "long",
        "defaultValue": 0,
        "skip": False,
        "details": {
            "directPull": True,
            "sourceEntity": "all_bookingpassenger",
            "sourceColumn": "PassengerID"
        }
    },
    {
        "name": "Revenue",
        "type": "double",
        "defaultValue": 0.0,
        "skip": False,
        "details": {
            "directPull": True,
            "sourceEntity": "all_bookingpassenger",
            "sourceColumn": "TotalCost"
        }
    },
    {
        "name": "BookingID",
        "type": "long",
        "defaultValue": 0,
        "skip": False,
        "details": {
            "directPull": True,
            "sourceEntity": "all_bookingpassenger",
            "sourceColumn": "BookingID"
        }
    },
    {
        "name": "TravelSoloOrGroup",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                # {"entityName": "all_passengertraveldoc", "columnName": "BookingID"},
            ],
            "function": "GetTravelSoloOrGroup"
        }
    },
    {
        "name": "PersonID",
        "type": "long",
        "defaultValue": 0,
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"}
            ],
            "function": "GetPersonID"
        }
    },
    {
        "name": "ProvisionalPrimaryKey",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"}
            ],
            "function": "GetProvisionalPrimaryKey"
        }
    },
    {
        "name": "IsRegistered",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "LastName"},
                {"entityName": "all_personname", "columnName": "FirstName"},
                {"entityName": "all_personname", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger",
                 "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetIsRegistered"
        }
    },
    {
        "name": "PaymentMethodCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_payment", "columnName": "PaymentMethodCode"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_payment", "columnName": "BookingID"},
            ],
            "function": "GetPaymentMethodCode"
        }
    },
    {
        "name": "FirstName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "FirstName"},
                {"entityName": "all_personname", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "MiddleName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "MiddleName"},
                {"entityName": "all_personname", "columnName": "MiddleName"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "LastName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "LastName"},
                {"entityName": "all_personname", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "HonorificPrefix",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "Title"},
                {"entityName": "all_personname", "columnName": "Title"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "HonorificSuffix",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "Suffix"},
                {"entityName": "all_personname", "columnName": "Suffix"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "EmailAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress",
                    "columnName": "EmailAddress"},
                {"entityName": "all_personemail", "columnName": "EMailAddress"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personemail", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personemail", "columnName": "PersonID"},

            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "Phone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress", "columnName": "Phone"},
                {"entityName": "all_personphone", "columnName": "Number"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personphone", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personphone", "columnName": "PersonID"},

            ],
            "function": "GetDetailsPhone"
        }
    },
    {
        "name": "City",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress", "columnName": "City"},
                {"entityName": "all_personaddress", "columnName": "City"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},

            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "State",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress",
                    "columnName": "ProvinceState"},
                {"entityName": "all_personaddress", "columnName": "ProvinceState"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "ZipCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress", "columnName": "PostalCode"},
                {"entityName": "all_personaddress", "columnName": "PostalCode"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "CountryCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengeraddress", "columnName": "CountryCode"},
                {"entityName": "all_personaddress", "columnName": "CountryCode"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "DateOfBirth",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "DOB"},
                {"entityName": "all_person", "columnName": "DOB"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },

    {
        "name": "Gender",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "Gender"},
                {"entityName": "all_person", "columnName": "Gender"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },
    {
        "name": "Nationality",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "Nationality"},
                {"entityName": "all_person", "columnName": "Nationality"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },

    {
        "name": "ResidentCountry",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger",
                    "columnName": "ResidentCountry"},
                {"entityName": "all_person", "columnName": "ResidentCountry"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },

    {
        "name": "CustomerNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger",
                    "columnName": "CustomerNumber"},
                {"entityName": "all_person", "columnName": "CustomerNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetDetails"
        }
    },

    {
        "name": "PassportNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "DocNumber"},
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "ExpirationDate"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "ExpirationDate"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetPassportNumber"
        }
    },

    {
        "name": "ExpiryDate",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc",
                    "columnName": "ExpirationDate"},
                {"entityName": "all_traveldoc", "columnName": "ExpirationDate"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "ExpirationDate"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "ExpirationDate"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetPassportNumber"
        }
    },

    {
        "name": "IssuingCountry",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc",
                    "columnName": "IssuedByCode"},
                {"entityName": "all_traveldoc", "columnName": "IssuedByCode"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "ExpirationDate"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "ExpirationDate"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetPassportNumber"
        }
    },

    {
        "name": "StreetAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_passengeraddress",
                    "columnName": "AddressLine1"},
                {"entityName": "all_passengeraddress",
                    "columnName": "AddressLine2"},
                {"entityName": "all_passengeraddress",
                    "columnName": "AddressLine3"},
                {"entityName": "all_personaddress", "columnName": "AddressLine1"},
                {"entityName": "all_personaddress", "columnName": "AddressLine2"},
                {"entityName": "all_personaddress", "columnName": "AddressLine3"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengeraddress", "columnName": "PassengerID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
            ],
            "function": "GetStreetAddress"
        }
    },

    {
        "name": "GetGoID",
        "type": "string",
        "defaultValue": "None",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "DocNumber"},
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetGoIdFun"
        }
    },

    {
        "name": "SeniorCitizenID",
        "type": "string",
        "defaultValue": "None",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "DocNumber"},
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetSeniorCitizenID"
        }
    },

    {
        "name": "BookedByAgentID",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
            ],
            "function": "GetBookedByAgentID"
        }
    },

    {
        "name": "TravelOrigin",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "DepartureStation"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "JourneyNumber"},

            ],
            "function": "GetTravelOriginAndDate"
        }
    },


    {
        "name": "TravelDate",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "DepartureDate"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "PassengerID"},

            ],
            "function": "GetTravelOriginAndDate"
        }
    },

    {
        "name": "TravelBaggage",
        "type": "string",
        "defaultValue": "None",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_fee", "columnName": "Name"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "FeeCode"},
                {"entityName": "all_fee", "columnName": "FeeCode"},

            ],
            "function": "GetTravelBaggageMealsInsurance"
        }
    },

    {
        "name": "TravelMeals",
        "type": "string",
        "defaultValue": "None",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_fee", "columnName": "Name"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "FeeCode"},
                {"entityName": "all_fee", "columnName": "FeeCode"},

            ],
            "function": "GetTravelBaggageMealsInsurance"
        }
    },

    {
        "name": "TravelInsurance",
        "type": "string",
        "defaultValue": "None",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_fee", "columnName": "Name"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "PassengerID"},
                {"entityName": "all_passengerfee", "columnName": "FeeCode"},
                {"entityName": "all_fee", "columnName": "FeeCode"},

            ],
            "function": "GetTravelBaggageMealsInsurance",
        }
    },

    {
        "name": "TravelSeat",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengerjourneyleg",
                    "columnName": "UnitDesignator"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneyleg",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneyleg",
                    "columnName": "ModifiedUTC"},
            ],
            "function": "GetTravelSeat",
        }
    },

    {
        "name": "TravelGetGoKeyedIn",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
            ],
            "function": "GetTravelGetGoKeyedIn",
        }
    },

    {
        "name": "IsPassenger",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
            ],
            "function": "GetIsPassenger",
        }
    },

    {
        "name": "IsEmployee",
        "type": "string",
        "defaultValue": "No",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "DocNumber"},
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetIsEmployee",
        }
    },

    {
        "name": "TravelDestination",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "ArrivalStation"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengerjourneysegment",
                    "columnName": "JourneyNumber"},

            ],
            "function": "GetTravelDestination",
        }
    },


    {
        "name": "IsEmployeeDependent",
        "type": "string",
        "defaultValue": "No",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_passengertraveldoc", "columnName": "FirstName"},
                {"entityName": "all_passengertraveldoc", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger", "columnName": "PassengerID"},
                {"entityName": "all_bookingpassenger", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "LastName"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "PassengerID"},
                {"entityName": "all_passengertraveldoc",
                    "columnName": "DocTypeCode"},
            ],
            "function": "GetIsEmployeeDependent",
        }
    },

    {
        "name": "BookerFirstName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerDetails
                {"entityName": "all_personname", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookerMiddleName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_personname", "columnName": "MiddleName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookerLastName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_personname", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "PersonID"},
                {"entityName": "all_personname", "columnName": "NameType"},
            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookerEmailAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerEmailAddress
                {"entityName": "all_personemail", "columnName": "EMailAddress"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personemail", "columnName": "PersonID"},
                {"entityName": "all_personemail", "columnName": "IsDefault"},

            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookingDate",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookingDate
                {"entityName": "all_booking", "columnName": "BookingUTC"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},

            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookingCurrency",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookingCurrency
                {"entityName": "all_booking", "columnName": "CurrencyCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},

            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookingChannel",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookingChannel
                {"entityName": "all_booking", "columnName": "ChannelType"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},

            ],
            "function": "GetBookingChannel",
        }
    },
    {
        "name": "BookedByPersonID",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookedByPersonID
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},

            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookerCity",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerCity
                {"entityName": "all_personaddress", "columnName": "City"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},

            ],
            "function": "GetBookerDetails",
        }
    },
    {
        "name": "BookerState",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerState
                {"entityName": "all_personaddress", "columnName": "ProvinceState"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
            ],
            "function": "GetBookerDetails",
        }
    },

    {
        "name": "BookerZipCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerZipCode
                {"entityName": "all_personaddress", "columnName": "PostalCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
            ],
            "function": "GetBookerDetails",
        }
    },

    {
        "name": "BookerCountryCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerCountryCode
                {"entityName": "all_personaddress", "columnName": "CountryCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
            ],
            "function": "GetBookerDetails",
        }
    },

    {
        "name": "BookerCustomerNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerCustomerNumber
                {"entityName": "all_person", "columnName": "CustomerNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetBookerDetails",
        }
    },


    {
        "name": "BookerPassportNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerPassportNumber
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetBookerPassportNumber",
        }
    },

    {
        "name": "BookerType",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerType
                {"entityName": "all_person", "columnName": "PersonType"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
            ],
            "function": "GetBookerType",
        }
    },

    {
        "name": "BookerMobile",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerMobile
                {"entityName": "all_personphone", "columnName": "Number"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personphone", "columnName": "PersonID"},
                {"entityName": "all_personphone", "columnName": "IsDefault"},
                {"entityName": "all_personphone", "columnName": "TypeCode"},
            ],
            "function": "GetBookerMobile",
        }
    },


    {
        "name": "BookerHomePhone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerHomePhone
                {"entityName": "all_personphone", "columnName": "Number"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personphone", "columnName": "PersonID"},
                {"entityName": "all_personphone", "columnName": "IsDefault"},
                {"entityName": "all_personphone", "columnName": "TypeCode"},
            ],
            "function": "GetBookerHomePhone",
        }
    },

    {
        "name": "BookerStreetAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerStreetAddress
                {"entityName": "all_personaddress", "columnName": "AddressLine1"},
                {"entityName": "all_personaddress", "columnName": "AddressLine2"},
                {"entityName": "all_personaddress", "columnName": "AddressLine3"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "PersonID"},
                {"entityName": "all_personaddress", "columnName": "IsDefault"},
            ],
            "function": "GetBookerStreetAddress",
        }
    },

    {
        "name": "BookerGetGoID",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                # GetBookerGetGoID
                {"entityName": "all_traveldoc", "columnName": "DocNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "BookingID"},
                {"entityName": "all_booking", "columnName": "CreatedUserID"},
                {"entityName": "all_agent", "columnName": "AgentID"},
                {"entityName": "all_agent", "columnName": "PersonID"},
                {"entityName": "all_person", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "PersonID"},
                {"entityName": "all_traveldoc", "columnName": "DocTypeCode"},
            ],
            "function": "GetBookerGetGoID",
        }
    },
    {
        "name": "PrimaryContactFirstName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactMiddleName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "MiddleName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },

    {
        "name": "PrimaryContactLastName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactEmail",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "EmailAddress"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactWorkPhone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "WorkPhone"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactHomePhone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "HomePhone"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactCity",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "City"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },

    {
        "name": "PrimaryContactState",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact",
                    "columnName": "ProvinceState"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactZipcode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "PostalCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactCountryCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "CountryCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "PrimaryContactCustomerNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact",
                    "columnName": "CustomerNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryNames",
        }
    },
    {
        "name": "AdditionalContactFirstName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "FirstName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactMiddleName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "MiddleName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },

    {
        "name": "AdditionalContactLastName",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "LastName"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactEmail",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "EmailAddress"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactWorkPhone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "WorkPhone"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactHomePhone",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "HomePhone"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactCity",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "City"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },

    {
        "name": "AdditionalContactState",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact",
                    "columnName": "ProvinceState"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactZipcode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "PostalCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactCountryCode",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact", "columnName": "CountryCode"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "AdditionalContactCustomerNumber",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [

                {"entityName": "all_bookingcontact",
                    "columnName": "CustomerNumber"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalNames",
        }
    },
    {
        "name": "PrimaryContactStreetAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingcontact", "columnName": "AddressLine1"},
                {"entityName": "all_bookingcontact", "columnName": "AddressLine2"},
                {"entityName": "all_bookingcontact", "columnName": "AddressLine3"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetPrimaryContactStreetAddress",
        }
    },
    {
        "name": "AdditionalContactStreetAddress",
        "type": "string",
        "defaultValue": "Unknown",
        "skip": False,
        "details": {
            "directPull": False,
            "sourceEntityDetails": [
                {"entityName": "all_bookingcontact", "columnName": "AddressLine1"},
                {"entityName": "all_bookingcontact", "columnName": "AddressLine2"},
                {"entityName": "all_bookingcontact", "columnName": "AddressLine3"},
                {"entityName": "all_bookingpassenger", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "BookingID"},
                {"entityName": "all_bookingcontact", "columnName": "TypeCode"},
            ],
            "function": "GetAdditionalContactStreetAddress",
        }
    },


                # GetIsRegistered and GetDetails
                # GetEmailAddress
                # GetInfoDetails
                # GetPassportNumber
                # GetStreetAddress
                # GetGoIdFun
                # GetSeniorCitizenId
                # GetBookedByAgentID
                # GetTravelOrigin
                # GetTravelDate
                # GetTravelBaggage
                # GetTravelMeals
                # GetTravelSeat
                # GetTravelGetGoKeyedIn
                # GetIsPassenger
                # GetIsEmployee
                # GetTravelDestination
                # GetIsEmployeeDependent
                # GetBookerDetails
                # GetBookerEmailAddress
                # GetBookingDate
                # GetBookingCurrency
                # GetBookingChannel
                # GetBookedByPersonID
                # GetBookerCity
                # GetBookerState
                # GetBookerZipCode
                # GetBookerCountryCode
                # GetBookerCustomerNumber
                # GetBookerPassportNumber
                # GetBookerType
                # GetBookerMobile
                # GetBookerHomePhone
                # GetBookerStreetAddress
                # GetBookerGetGoID
                # GetPrimaryNames
                # GetAdditionalNames
                # GetPrimaryContactStreetAddress
                # GetAdditionalContactStreetAddress
      
    # {'name': 'FirstName', 'type': 'string', 'skip': True},
    # {'name': 'MiddleName', 'type': 'string', 'skip': True},
    # {'name': 'LastName', 'type': 'string', 'skip': True},
    # {'name': 'HonorificPrefix', 'type': 'string', 'skip': True},
    # {'name': 'HonorificSuffix', 'type': 'string', 'skip': True},
    # {'name': 'EmailAddress', 'type': 'string', 'skip': True},
    # {'name': 'Phone', 'type': 'string', 'skip': True},
    # {'name': 'City', 'type': 'string', 'skip': True},
    # {'name': 'State', 'type': 'string', 'skip': True},
    # {'name': 'ZipCode', 'type': 'string', 'skip': True},
    # {'name': 'CountryCode', 'type': 'string', 'skip': True},
    # {'name': 'DateOfBirth', 'type': 'string', 'skip': True},
    # {'name': 'Gender', 'type': 'string', 'skip': True},
    # {'name': 'Nationality', 'type': 'string', 'skip': True},
    # {'name': 'ResidentCountry', 'type': 'string', 'skip': True},
    # {'name': 'CustomerNumber', 'type': 'string', 'skip': True},
    # {'name': 'PassportNumber', 'type': 'string', 'skip': True},
    # {'name': 'ExpiryDate', 'type': 'string', 'skip': True},
    # {'name': 'IssuingCountry', 'type': 'string', 'skip': True},
    # {'name': 'StreetAddress', 'type': 'string', 'skip': True},
    # {'name': 'GetGoID', 'type': 'string', 'skip': True},
    # {'name': 'SeniorCitizenID', 'type': 'string', 'skip': True},
    # {'name': 'BookedByAgentID', 'type': 'string', 'skip': True},
    # {'name': 'TravelOrigin', 'type': 'string', 'skip': True},
    # {'name': 'TravelDate', 'type': 'string', 'skip': True},
    # {'name': 'TravelBaggage', 'type': 'string', 'skip': True},
    # {'name': 'TravelMeals', 'type': 'string', 'skip': True},
    # {'name': 'TravelSeat', 'type': 'string', 'skip': True},
    # {'name': 'TravelInsurance', 'type': 'string', 'skip': True},
    # {'name': 'TravelGetGoKeyedIn', 'type': 'string', 'skip': True},
    # {'name': 'IsPassenger', 'type': 'string', 'skip': True},
    # {'name': 'IsEmployee', 'type': 'string', 'skip': True},
    # {'name': 'BookerFirstName', 'type': 'string', 'skip': True},
    # {'name': 'BookerMiddleName', 'type': 'string', 'skip': True},
    # {'name': 'BookerLastName', 'type': 'string', 'skip': True},
    # {'name': 'BookerEmailAddress', 'type': 'string', 'skip': True},
    # {'name': 'TravelDestination', 'type': 'string', 'skip': True},
    # {'name': 'IsEmployeeDependent', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactFirstName', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactMiddleName', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactLastName', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactEmail', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactWorkPhone', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactHomePhone', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactCity', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactState', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactZipcode', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactCountryCode', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactCustomerNumber', 'type': 'string', 'skip': True},
    # {'name': 'PrimaryContactStreetAddress', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactFirstName', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactMiddleName', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactLastName', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactEmail', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactWorkPhone', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactHomePhone', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactCity', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactState', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactZipcode', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactCountryCode', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactCustomerNumber', 'type': 'string', 'skip': True},
    # {'name': 'AdditionalContactStreetAddress', 'type': 'string', 'skip': True},
    # {'name': 'BookingDate', 'type': 'string', 'skip': True},
    # {'name': 'BookingCurrency', 'type': 'string', 'skip': True},
    # {'name': 'BookingChannel', 'type': 'string', 'skip': True},
    # {'name': 'BookedByPersonID', 'type': 'long', 'skip': True},
    # {'name': 'BookerType', 'type': 'string', 'skip': True},
    # {'name': 'BookerMobile', 'type': 'string', 'skip': True},
    # {'name': 'BookerHomePhone', 'type': 'string', 'skip': True},
    # {'name': 'BookerStreetAddress', 'type': 'string', 'skip': True},
    # {'name': 'BookerCity', 'type': 'string', 'skip': True},
    # {'name': 'BookerState', 'type': 'string', 'skip': True},
    # {'name': 'BookerZipCode', 'type': 'string', 'skip': True},
    # {'name': 'BookerCountryCode', 'type': 'string', 'skip': True},
    # {'name': 'BookerPassportNumber', 'type': 'string', 'skip': True},
    # {'name': 'BookerGetGoID', 'type': 'string', 'skip': True},
    # {'name': 'BookerCustomerNumber', 'type': 'string', 'skip': True}
]
profile_config["profileSchema"], profile_config["columnMapping"], profile_config["attributeMapping"] = columnCompute(
    columnDetails=columnDetails)


In [4]:
!pip3 install pandas
import pandas as pd
# df_seed = pd.read_csv("seed.csv")
# # df_seed['column'] to list
# all_columns = []
# #  remove spaces in each item string of the list all_columns
# for i in df_seed['column'].tolist():
#     i = i.replace(" ", "")
#     all_columns.append(i)

# len(columnDetails)
# save columnDetails keys to an array
# columnDetailsKeys = []

# for i in range(len(columnDetails)):

#     columnDetailsKeys.append(columnDetails[i]['name'])
# # check what columns in all_columns are missing in columnDetailsKeys
# missing_columns = [x for x in all_columns if x not in columnDetailsKeys]
# # print missing columns
# print(missing_columns)


In [ ]:
def storageInfo(profilePath = "s3a://cebu-cdp-data-dev/profiles/master",profileMetaPath = "s3a://cebu-cdp-metadata-dev/last_modified_records/master", dedupePath = "s3a://cebu-cdp-data-dev/profiles/master/master_profile_delta",ucpProfilePath = "s3a://cebu-cdp-data-dev/profiles/master/master_profile_delta",uiPath = ""):
    return  [
    {
      "recordType": "valid-records",
      "dataFormat": "parquet",
      "fileSystem": "s3",
      "pathUrl": profilePath,
      "storageHandler": "PersistHadoopFS"
    },
    {
      "recordType": "dedupe",
      "dataFormat": "parquet",
      "fileSystem": "s3",
      "pathUrl": dedupePath,
      "storageHandler": "PersistHadoopFS"
    },
    {
      "recordType": "ucp",
      "dataFormat": "parquet",
      "fileSystem": "s3",
      "pathUrl": ucpProfilePath,
      "storageHandler": "PersistHadoopFS"
    },
    {
      "recordType": "ui",
      "dataFormat": "parquet",
      "fileSystem": "s3",
      "pathUrl": uiPath,
      "storageHandler": "PersistHadoopFS"
    }
    # {
    #   "recordType": "valid-records",
    #   "dataFormat": "csv",
    #   "dataFormatOption": [
    #     {
    #       "key": "codec",
    #       "value": "org.apache.hadoop.io.compress.GzipCodec"
    #     },
    #     {
    #       "key": "timestampFormat",
    #       "value": "yyyy-MM-dd HH:mm:ss"
    #     },
    #     {
    #       "key": "dateFormat",
    #       "value": "yyyy-MM-dd"
    #     },
    #     {
    #       "key": "escape",
    #       "value": "\""
    #     }
    #   ],
    #   "fileSystem": "s3",
    #   "pathUrl": profilePath+"/master_profile_csv",
    #   "storageHandler": "PersistHadoopFS"
    # },
    # {
    #   "recordType": "last-modified-records",
    #   "dataFormat": "parquet",
    #   "fileSystem": "s3",
    #   "pathUrl": profileMetaPath+"/master_profile",
    #   "storageHandler": "PersistHadoopFS"
    # }
  ]
profile_config['storageDetails'] = storageInfo(profilePath=profilePath,profileMetaPath=profileMetaPath, dedupePath=dedupePath, ucpProfilePath=ucpProfilePath, uiPath=uiPath)

In [ ]:
profile_config['ciamPath'] = ciamPath

In [ ]:
# save profile_config to json file
# import json
# with open('src/test/resources/configs/profile_configs/profile_config_cebu.json', 'w') as outfile:
#     json.dump(profile_config, outfile)

# save profile_config to json file
import json
with open('profile_config_cebu_v1.json', 'w') as outfile:
    json.dump(profile_config, outfile)

In [ ]:
# profile_config['attributeMapping'][37]